In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import model_from_json
from sklearn.metrics import classification_report, confusion_matrix

Load data citra yang telah disimpan pada file .csv. Data yang terdapat pada file telah diubah dalam bentuk vektor dimana baris merupakan representasi banyak citra dan kolom merepresentasi panjang citra pada x_train dan x_test.

In [2]:
x_train= np.genfromtxt('x_train.csv', delimiter=',')
y_train = np.genfromtxt('y_train.csv', delimiter=',')
x_test = np.genfromtxt('x_test.csv', delimiter=',')
y_test = np.genfromtxt('y_test.csv', delimiter=',')

Lakukan normalisasi pada data dengan cara membagi data dengan angka 255 yang merupakan nilai maksimal suatu piksel pada citra. Ubah data latih dan data uji menjadi array 4 dimensi (Tensor) agar bisa dimuat pada model TensorFlow. Input shape disesuaikan dengan ukuran citra, dimana untuk data citra berukuran 100x100 piksel.

In [3]:
x_train, x_test = x_train/255.0, x_test/255.0
x_train = x_train.reshape(x_train.shape[0], 100, 100, 1)
x_test = x_test.reshape(x_test.shape[0], 100, 100, 1)
input_shape = (100,100, 1)


Model CNN yang digunakan menggunakan satu layer konvolusi dengan filter 3x3 berjumlah 28 dan dua fully connected layer dengan jumlah node 512 dan 10. Fully connected layer dilengkapi dengan dropout untuk menghindari overfitting.Pada layer terakhir digunakan fungsi aktivasi softmax yang akan menghasilkan probabilitas dari tiap node di output. Model dikompilasi dengan adam optimizer dan sparse categorical crossentropy loss function.

In [4]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(28, kernel_size = (3, 3), input_shape = input_shape),
    tf.keras.layers.MaxPooling2D(pool_size = (2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation = tf.nn.relu),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation = tf.nn.softmax)
])

In [5]:
model.compile(optimizer = 'sgd',
             loss = 'sparse_categorical_crossentropy',
             metrics = ['accuracy'])

In [6]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs = 100,verbose=2)

Train on 40 samples, validate on 20 samples
Epoch 1/100
 - 3s - loss: 2.1996 - acc: 0.2250 - val_loss: 1.6071 - val_acc: 0.5000
Epoch 2/100
 - 1s - loss: 1.5379 - acc: 0.5500 - val_loss: 1.3920 - val_acc: 0.5000
Epoch 3/100
 - 1s - loss: 1.3021 - acc: 0.6000 - val_loss: 1.2603 - val_acc: 0.5000
Epoch 4/100
 - 1s - loss: 1.1975 - acc: 0.5500 - val_loss: 1.1215 - val_acc: 0.4500
Epoch 5/100
 - 1s - loss: 1.0300 - acc: 0.5750 - val_loss: 1.0135 - val_acc: 0.7500
Epoch 6/100
 - 1s - loss: 0.8869 - acc: 0.8250 - val_loss: 1.0433 - val_acc: 0.5000
Epoch 7/100
 - 1s - loss: 0.9269 - acc: 0.5250 - val_loss: 0.9077 - val_acc: 0.5500
Epoch 8/100
 - 1s - loss: 0.7248 - acc: 0.8250 - val_loss: 1.5333 - val_acc: 0.5000
Epoch 9/100
 - 1s - loss: 1.1761 - acc: 0.6000 - val_loss: 0.7372 - val_acc: 0.9000
Epoch 10/100
 - 1s - loss: 0.6364 - acc: 0.9250 - val_loss: 0.6950 - val_acc: 0.9000
Epoch 11/100
 - 1s - loss: 0.6157 - acc: 0.7500 - val_loss: 1.0330 - val_acc: 0.5000
Epoch 12/100
 - 1s - loss: 0.8

Epoch 97/100
 - 1s - loss: 0.0366 - acc: 1.0000 - val_loss: 0.1215 - val_acc: 0.9500
Epoch 98/100
 - 1s - loss: 0.0326 - acc: 1.0000 - val_loss: 0.1361 - val_acc: 0.9500
Epoch 99/100
 - 1s - loss: 0.0451 - acc: 1.0000 - val_loss: 0.1570 - val_acc: 0.9500
Epoch 100/100
 - 1s - loss: 0.0294 - acc: 1.0000 - val_loss: 0.1841 - val_acc: 0.9000


In [8]:
model.evaluate(x_test, y_test)

20/20 [==============================] - 0s 6ms/step


[0.1841282695531845, 0.8999999761581421]

## Peforma Sistem
Performa sistem divisualisasikan menggunakan confusion matrix. Ada tiga score yang diperlihatkan, yaitu precision, recall, dan f1-score. Nilai precision menunjukkan akurasi model dalam mengklasifikasi data yang ada di kelas diuji, sedangkan nilai recall menunjukkan akurasi mode dalam mengklasifikasi data yang bukan dari kelas yang diuji. Nilai f1-score menunjukkan akurasi model secara keseluruhan.

In [9]:
predicted=model.predict_classes(x_test)
print("Classification report for classifier %s:\n%s\n"
      % (model, classification_report(y_test, predicted)))
print("Confusion matrix:\n%s" % confusion_matrix(y_test, predicted))

Classification report for classifier <tensorflow.python.keras.engine.sequential.Sequential object at 0x0000022BCBD3C5F8>:
             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00         5
        1.0       1.00      1.00      1.00         5
        2.0       1.00      0.60      0.75         5
        3.0       0.71      1.00      0.83         5

avg / total       0.93      0.90      0.90        20


Confusion matrix:
[[5 0 0 0]
 [0 5 0 0]
 [0 0 3 2]
 [0 0 0 5]]


## Proses Penyimpanan Model dan Weight

In [10]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
